# Prate Individual netCDF Files for Each Variable (mean, clim, anom,std)

In [1]:
import os
import pandas as pd
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib
import datetime
from dateutil.relativedelta import relativedelta

# prate

In [4]:
os.chdir('d:/NCEP_R2')
ds_prate = xr.open_dataset('prate.sfc.mon.mean.nc') # ds_precip

### cut new clim_prate
- prate
- clim_ds
- anom_ds
- std_ds

In [16]:
new_clim = ds_prate.sel(time=slice('1991-01-01','2020-12-01'))
new_clim

<xarray.Dataset>
Dimensions:  (time: 360, lat: 94, lon: 192)
Coordinates:
  * time     (time) datetime64[ns] 1991-01-01 1991-02-01 ... 2020-12-01
  * lon      (lon) float32 0.0 1.875 3.75 5.625 7.5 ... 352.5 354.4 356.2 358.1
  * lat      (lat) float32 88.54 86.65 84.75 82.85 ... -84.75 -86.65 -88.54
Data variables:
    prate    (time, lat, lon) float32 ...

In [17]:
# unit : Kg/m^2/s -> mm/mon
for i in range(len(new_clim.prate)):
    p_time = str(new_clim.time[i].values)[:10]
    b = datetime.datetime.strptime(p_time,'%Y-%m-%d')
    num_of_day = ((b + relativedelta(months=i))-datetime.timedelta(days=1)).day
    sec_to_month = 60*60*24* num_of_day
    new_clim.prate[i] *= sec_to_month
prate = new_clim.prate

lon = new_clim.lon
lat = new_clim.lat
time = new_clim.time

#### ◆ prate nc file (monthly mean)

In [22]:
prate_arr = np.array(prate)

prate_ds = xr.Dataset(
    data_vars = dict(
        prate = (['time','lat','lon'],prate_arr)
    ),
    coords = dict(
        lon = lon,
        lat = lat,
        time = time
    )
)

prate_ds.lon.attrs = {'units':'degree_east','long_name':'Longitude'}
prate_ds.lat.attrs = {'units':'degree_north','long_name': 'Latitude'}
prate_ds.prate.attrs ={'units':'mm/month','long_name':'Monthly Mean of Precipitation Rate at Surface'}


In [24]:
prate_ds.to_netcdf('8.NCEP_ncfile(mean,clim,anom,std)/individual_ncfile/NCEP_prate_sfc_mon_mean_new.nc')

#### ◆ prate clim nc file

In [25]:
clim_sel = new_clim['prate'].groupby('time.month').mean('time')

clim_extend_lst = []
for i in range(30):
    for j in clim_sel.values:
        clim_extend_lst.append(j)
        
clim_extend_arr = np.array(clim_extend_lst)
        
clim_ds = xr.Dataset(
    data_vars = dict(
        new_clim = (['time','lat','lon'], clim_extend_arr)
    ),
    coords = dict(
        lon = lon,
        lat = lat,
        time = time
    )
)

In [27]:
## apply attributes
clim_ds.lon.attrs = {'units':'degree_east','long_name':'Longitude'}
clim_ds.lat.attrs = {'units':'degree_north','long_name': 'Latitude'}
clim_ds.new_clim.attrs ={'units':'mm/month','long_name':'Average Long Term Mean Monthly Rate of Precipitation at Surface'}

In [28]:
# os.getcwd()
clim_ds.to_netcdf('8.NCEP_ncfile(mean,clim,anom,std)/individual_ncfile/NECP_prate_sfc_clim_new.nc')

#### ◆ prate anom nc file

In [81]:
anom_lst = []
for i in range(1,13):
    each_mon = new_clim['prate'].sel(time=new_clim['time.month']==i) # month
    # clim_mon = clim_sel.sel(month=i) # clim # new_clim['prate'].groupby('time.month').mean('time')
    clim_mon = clim_ds['new_clim'].sel(time=clim_ds['time.month']==i) 
    
    anom_sel = each_mon - clim_mon # anom
    anom_lst.append(anom_sel) 
    
anom_cb  = xr.concat(anom_lst, dim='time')

anom_ds = xr.Dataset(
    data_vars = dict(
        new_anom = anom_cb
        ),
    coords = dict(
        lon = lon,
        lat = lat,
        time = time
    )
)


c:\Users\ocpc\.conda\envs\cartopy\lib\site-packages\xarray\core\concat.py:500: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  common_dims = tuple(pd.unique([d for v in vars for d in v.dims]))


In [85]:
## apply attributes
anom_ds.lon.attrs = new_clim.lon.attrs
anom_ds.lat.attrs = {'units':'degree_north','long_name': 'Latitude'}
anom_ds.new_anom.attrs ={'units':'mm/month','long_name':'Anomaly Monthly Rate of Precipitation at Surface'} 
anom_ds

<xarray.Dataset>
Dimensions:   (time: 360, lon: 192, lat: 94)
Coordinates:
  * time      (time) datetime64[ns] 1991-01-01 1991-02-01 ... 2020-12-01
  * lon       (lon) float32 0.0 1.875 3.75 5.625 7.5 ... 352.5 354.4 356.2 358.1
  * lat       (lat) float32 88.54 86.65 84.75 82.85 ... -84.75 -86.65 -88.54
Data variables:
    new_anom  (time, lat, lon) float32 -5.732 -5.527 -5.589 ... 4.491 4.343

In [84]:
os.getcwd()
# anom_ds.to_netcdf('8.NCEP_ncfile(mean,clim,anom,std)/individual_ncfile/NECP_prate_sfc_anom_new.nc')

#### ◆ prate std nc file

In [86]:
## std
std_sel=new_clim.prate.groupby('time.month').std(dim='time',ddof=1)

## Change the moth dim to the time dim
std_extend_lst = []

for i in range(30):
    for j in std_sel.values:
            std_extend_lst.append(j)

std_extend_arr = np.array(std_extend_lst)
        
std_ds = xr.Dataset(
    data_vars=dict(
        new_std = (['time','lat','lon'],std_extend_arr)
    ),
    coords= dict(
        lon = lon,
        lat = lat,
        time = time
    )
)       

## apply attributes
std_ds.lon.attrs = {'units':'degree_east','long_name':'Longitude'}
std_ds.lat.attrs = {'units':'degree_north','long_name': 'Latitude'}
std_ds.new_std.attrs ={'unit':'mm/month','long_name':'Standard Deviation Rate of Precipitation'}

In [87]:
os.getcwd()
# std_ds.to_netcdf('8.NCEP_ncfile(mean,clim,anom,std)/individual_ncfile/prate/NECP_prate_sfc_std_new.nc')

### cut old clim_prate
- prate
- clim_ds
- anom_ds
- std_ds

In [89]:
old_clim =  ds_prate.sel(time=slice('1981-01-01','2010-12-01'))
old_clim

<xarray.Dataset>
Dimensions:  (time: 360, lat: 94, lon: 192)
Coordinates:
  * time     (time) datetime64[ns] 1981-01-01 1981-02-01 ... 2010-12-01
  * lon      (lon) float32 0.0 1.875 3.75 5.625 7.5 ... 352.5 354.4 356.2 358.1
  * lat      (lat) float32 88.54 86.65 84.75 82.85 ... -84.75 -86.65 -88.54
Data variables:
    prate    (time, lat, lon) float32 ...

In [90]:
# unit : Kg/m^2/s -> mm/mon
for i in range(len(old_clim.prate)):
    p_time = str(old_clim.time[i].values)[:10]
    b = datetime.datetime.strptime(p_time,'%Y-%m-%d')
    num_of_day = ((b + relativedelta(months=i))-datetime.timedelta(days=1)).day
    sec_to_month = 60*60*24* num_of_day
    old_clim.prate[i] *= sec_to_month
prate = old_clim.prate

lon = old_clim.lon
lat = old_clim.lat
time = old_clim.time

#### ◆ prate nc file (monthly mean)

In [92]:
prate_arr = np.array(prate)

prate_ds = xr.Dataset(
    data_vars = dict(
        prate = (['time','lat','lon'],prate_arr)
    ),
    coords = dict(
        lon = lon,
        lat = lat,
        time = time
    )
)

prate_ds.lon.attrs = {'units':'degree_east','long_name':'Longitude'}
prate_ds.lat.attrs = {'units':'degree_north','long_name': 'Latitude'}
prate_ds.prate.attrs ={'units':'mm/month','long_name':'Monthly Mean of Precipitation Rate at Surface'}

# prate_ds.to_netcdf('8.NCEP_ncfile(mean,clim,anom,std)/individual_ncfile/NCEP_prate_sfc_mon_mean_old.nc')

#### ◆ prate clim nc file

In [94]:
clim_sel = old_clim['prate'].groupby('time.month').mean('time')

clim_extend_lst = []
for i in range(30):
    for j in clim_sel.values:
        clim_extend_lst.append(j)
        
clim_extend_arr = np.array(clim_extend_lst)
        
clim_ds = xr.Dataset(
    data_vars = dict(
        old_clim = (['time','lat','lon'], clim_extend_arr)
    ),
    coords = dict(
        lon = lon,
        lat = lat,
        time = time
    )
)

## apply attributes
clim_ds.lon.attrs = {'units':'degree_east','long_name':'Longitude'}
clim_ds.lat.attrs = {'units':'degree_north','long_name': 'Latitude'}
clim_ds.old_clim.attrs ={'units':'mm/month','long_name':'Average Long Term Mean Monthly Rate of Precipitation at Surface'}

# os.getcwd()
# clim_ds.to_netcdf('8.NCEP_ncfile(mean,clim,anom,std)/individual_ncfile/NECP_prate_sfc_clim_old.nc')

#### ◆ prate anom nc file

In [95]:
anom_lst = []
for i in range(1,13):
    each_mon = old_clim['prate'].sel(time=old_clim['time.month']==i) # month
    # clim_mon = clim_sel.sel(month=i) # clim # old_clim['prate'].groupby('time.month').mean('time')
    clim_mon = clim_ds['old_clim'].sel(time=clim_ds['time.month']==i) 
    
    anom_sel = each_mon - clim_mon # anom
    anom_lst.append(anom_sel) 
    
anom_cb  = xr.concat(anom_lst, dim='time')

anom_ds = xr.Dataset(
    data_vars = dict(
        old_anom = anom_cb
        ),
    coords = dict(
        lon = lon,
        lat = lat,
        time = time
    )
)
## apply attributes
anom_ds.lon.attrs = new_clim.lon.attrs
anom_ds.lat.attrs = {'units':'degree_north','long_name': 'Latitude'}
anom_ds.old_anom.attrs ={'units':'mm/month','long_name':'Anomaly Monthly Rate of Precipitation at Surface'} 
anom_ds

c:\Users\ocpc\.conda\envs\cartopy\lib\site-packages\xarray\core\concat.py:500: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  common_dims = tuple(pd.unique([d for v in vars for d in v.dims]))


<xarray.Dataset>
Dimensions:   (time: 360, lon: 192, lat: 94)
Coordinates:
  * time      (time) datetime64[ns] 1981-01-01 1981-02-01 ... 2010-12-01
  * lon       (lon) float32 0.0 1.875 3.75 5.625 7.5 ... 352.5 354.4 356.2 358.1
  * lat       (lat) float32 88.54 86.65 84.75 82.85 ... -84.75 -86.65 -88.54
Data variables:
    old_anom  (time, lat, lon) float32 -4.107 -4.348 -4.357 ... -1.687 -1.634

In [96]:
os.getcwd()
anom_ds.to_netcdf('8.NCEP_ncfile(mean,clim,anom,std)/individual_ncfile/NECP_prate_sfc_anom_old.nc')

#### ◆ prate std nc file

In [97]:
## std
std_sel=new_clim.prate.groupby('time.month').std(dim='time',ddof=1)

## Change the moth dim to the time dim
std_extend_lst = []

for i in range(30):
    for j in std_sel.values:
            std_extend_lst.append(j)

std_extend_arr = np.array(std_extend_lst)
        
std_ds = xr.Dataset(
    data_vars=dict(
        old_std = (['time','lat','lon'],std_extend_arr)
    ),
    coords= dict(
        lon = lon,
        lat = lat,
        time = time
    )
)       

## apply attributes
std_ds.lon.attrs = {'units':'degree_east','long_name':'Longitude'}
std_ds.lat.attrs = {'units':'degree_north','long_name': 'Latitude'}
std_ds.old_std.attrs ={'unit':'mm/month','long_name':'Standard Deviation Rate of Precipitation'}

In [98]:
os.getcwd()
std_ds.to_netcdf('8.NCEP_ncfile(mean,clim,anom,std)/individual_ncfile/prate/NECP_prate_sfc_std_old.nc')